Get all Swedish cemeteries and check OSM
* [This Notebook](https://github.com/salgo60/Gravstensinventeringen-Wikidata/blob/main/Notebook/OSM%20Wikidata%20kyrkog%C3%A5rd.ipynb)
* [Video](https://youtu.be/EDkPoomRmJg)


In [1]:
from datetime import datetime
start_time  = datetime.now()
print("Last run: ", start_time)

Last run:  2023-01-13 14:23:30.714440


In [2]:
import pandas as pd
#
# pip install sparqlwrapper
# https://rdflib.github.io/sparqlwrapper/

import sys,json
from SPARQLWrapper import SPARQLWrapper, JSON

endpoint_url = "https://query.wikidata.org/sparql"
 
# https://w.wiki/6Dgz
query = """SELECT (REPLACE(STR(?item), ".*Q", "Q") AS ?itemid) ?itemLabel ?OSM ?OSMrel WHERE {
  ?item wdt:P6104 wd:Q115206846.
  ?item wdt:P17 wd:Q34.
  OPTIONAL {?item wdt:P10689 ?OSM}
  OPTIONAL {?item wdt:P402 ?OSMrel}


  SERVICE wikibase:label { bd:serviceParam wikibase:language "sv,en". }
}
ORDER BY (?itemLabel)"""


def get_sparql_dataframe(endpoint_url, query):
    """
    Helper function to convert SPARQL results into a Pandas data frame.
    """
    user_agent = "salgo60/%s.%s" % (sys.version_info[0], sys.version_info[1])
 
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    result = sparql.query()

    processed_results = json.load(result.response)
    cols = processed_results['head']['vars']

    out = []
    for row in processed_results['results']['bindings']:
        item = []
        for c in cols:
            item.append(row.get(c, {}).get('value'))
        out.append(item)

    return pd.DataFrame(out, columns=cols)

WDSwe = get_sparql_dataframe(endpoint_url, query)    
WDSwe.shape

(3489, 4)

In [3]:
WDSwe.head()

,itemid,itemLabel,OSM,OSMrel
0,Q115961202,Lindhagens kulle,None,None
1,Q115302075,Abilds kyrkogård,way/1116387174,None
2,Q115371417,Acklinga kyrkogård,None,6363071
3,Q115364976,Adak kyrkogård,way/839880966,None
4,Q91411700,Adelsö kyrkogård,way/391824428,None


In [4]:
import urllib3, json
from tqdm import tqdm
http = urllib3.PoolManager()

listWD = []
for WD, row in tqdm(WDSwe.iterrows(), total=WDSwe.shape[0]):
    #print(row["itemid"] ) 
    url = "https://osm.wikidata.link/tagged/api/item/" + row["itemid"] 
    
    new_item = dict()
    new_item['wikidata'] = row["itemid"] 
#    new_item['coord'] = row["coord"] 
    try:
        r = http.request('GET', url) 
        data = json.loads(r.data.decode('utf-8'))
    except:
        print (r.status, url)
#    print (r.status)
    try:
        #print ("OSM ", data["osm"], "Type: ", type(data["osm"]))
        #print ("ID: ", data["osm"][0]["id"] , "\tType: ", data["osm"][0]["type"])
        osmid = data["osm"][0]["id"] 
        osmType = data["osm"][0]["type"]
        new_item['osmid'] = osmid 
        new_item['type'] = osmType
        listWD.append(new_item)
        #print(listWD)

    except:
        #print ("error")
        pass
print (len(listWD))

 22%|██▏       | 772/3489 [01:11<03:50, 11.79it/s]

404 https://osm.wikidata.link/tagged/api/item/http://www.wikidata.org/entity/P5259


100%|██████████| 3489/3489 [05:22<00:00, 10.83it/s]

149


In [5]:
listWD

[{'wikidata': 'Q91411700', 'osmid': 391824428, 'type': 'way'},
 {'wikidata': 'Q360572', 'osmid': 42038493, 'type': 'way'},
 {'wikidata': 'Q19781340', 'osmid': 15802789, 'type': 'way'},
 {'wikidata': 'Q105451716', 'osmid': 319722710, 'type': 'way'},
 {'wikidata': 'Q50821184', 'osmid': 906518009, 'type': 'way'},
 {'wikidata': 'Q105521284', 'osmid': 6386650, 'type': 'relation'},
 {'wikidata': 'Q10428784', 'osmid': 32053420, 'type': 'way'},
 {'wikidata': 'Q19977506', 'osmid': 278976961, 'type': 'way'},
 {'wikidata': 'Q91453681', 'osmid': 609488819, 'type': 'way'},
 {'wikidata': 'Q91654592', 'osmid': 198936254, 'type': 'way'},
 {'wikidata': 'Q10432963', 'osmid': 240376988, 'type': 'way'},
 {'wikidata': 'Q91413168', 'osmid': 188549828, 'type': 'way'},
 {'wikidata': 'Q44436044', 'osmid': 312365242, 'type': 'way'},
 {'wikidata': 'Q19672911', 'osmid': 43395692, 'type': 'way'},
 {'wikidata': 'Q91422098', 'osmid': 236563790, 'type': 'way'},
 {'wikidata': 'Q19721229', 'osmid': 515299597, 'type': '

In [6]:
OSMtot = pd.DataFrame(listWD,
                  columns=['wikidata','osmid','type'])
OSMtot.shape

(149, 3)

In [7]:
pd.set_option('max_colwidth', 400)
OSMtot.head(10)

,wikidata,osmid,type
0,Q91411700,391824428,way
1,Q360572,42038493,way
2,Q19781340,15802789,way
3,Q105451716,319722710,way
4,Q50821184,906518009,way
5,Q105521284,6386650,relation
6,Q10428784,32053420,way
7,Q19977506,278976961,way
8,Q91453681,609488819,way
9,Q91654592,198936254,way


In [8]:
OSMtot.to_csv("WDOSMCemetary.csv")

* [WDOSMCemetary.csv](https://github.com/salgo60/Gravstensinventeringen-Wikidata/blob/main/Notebook/WDOSMCemetary.csv)

In [9]:
end = datetime.now()
print("Ended: ", end) 
print('Time elapsed (hh:mm:ss.ms) {}'.format(datetime.now() - start_time))

Ended:  2023-01-13 14:29:04.289034
Time elapsed (hh:mm:ss.ms) 0:05:33.575296
